In [1]:
import numpy as np
import pandas as pd
from skimage import io, measure
from pathlib import Path


In [ ]:
import pandas as pd
from pathlib import Path
from skimage import io, measure

# 1️⃣ Load labels
labels_path = r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Is It This One\data\labels.csv"
labels_df = pd.read_csv(labels_path)

# 2️⃣ Extract shape features
mask_dir = Path(r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Is It This One\data\predicted_masks")
mask_paths = sorted(mask_dir.glob("*.png"))

features_list = []

for mask_path in mask_paths:
    mask = io.imread(mask_path, as_gray=True)
    mask = (mask > 127).astype(int)  # binary mask

    labeled_mask = measure.label(mask)
    regions = measure.regionprops(labeled_mask)

    if len(regions) == 0:
        continue

    # take the largest connected component
    region = max(regions, key=lambda r: r.area)

    features = {
        "id": int(mask_path.stem.split("_")[0]),
        "area": region.area,
        "perimeter": region.perimeter,
        "eccentricity": region.eccentricity,
        "solidity": region.solidity,
        "extent": region.extent,
        "major_axis_length": region.major_axis_length,
        "minor_axis_length": region.minor_axis_length,
        "orientation": region.orientation
    }
    features_list.append(features)

shape_df = pd.DataFrame(features_list)

# 3️⃣ Merge with labels
merged_df = pd.merge(labels_df, shape_df, on="id", how="inner")

# 4️⃣ Save CSV
merged_df.to_csv(r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Is It This One\data\features_with_labels.csv", index=False)

print("Shape features merged with labels and saved as features_with_labels.csv")


Shape features merged with labels and saved as features_with_labels.csv


In [9]:
import zipfile
from pathlib import Path
import os

# Folders to compress
folders_to_zip = [
    "data/predicted_masks",
    "checkpoints"
]

for folder in folders_to_zip:
    folder_path = Path(folder)
    zip_name = folder_path.name + ".zip"

    print(f"Compressing {folder} → {zip_name}")
    with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        for file_path in folder_path.rglob("*"):
            zipf.write(file_path, arcname=file_path.relative_to(folder_path.parent))

    # Remove folder from Git tracking but keep locally
    os.system(f"git rm -r --cached {folder}")

    # Add the zip file to Git
    os.system(f"git add {zip_name}")

print("Compression done! Now commit and push:")
print("git commit -m 'Compress large folders for faster push'")
print("git push -u origin <branch_name>")


Compressing data/predicted_masks → predicted_masks.zip
Compressing checkpoints → checkpoints.zip
Compression done! Now commit and push:
git commit -m 'Compress large folders for faster push'
git push -u origin <branch_name>
